In [1]:
using JuMP, Gurobi
using LinearAlgebra
using Distributions, Random, PDMats
using MLJ, Tables
using DataFrames, DataFramesMeta
using JLD, CSV
using PlotlyJS
using Pipe
using LaTeXStrings

Random.seed!(42)

TaskLocalRNG()

In [2]:
using Revise

includet("../models/forward.jl")
import .Forward as Forward

includet("../models/inversedemand.jl")
import .InverseDemand as IODemand

includet("../models/inverselinreg.jl")
import .InverseLinReg as IOLinReg

includet("../datagen/data-generation.jl")
import .DataGeneration as DataGen

In [71]:
LinearW = @MLJ.load LinearRegressor pkg = "GLM"

import MLJGLMInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/luca/.julia/packages/MLJModels/Ft0z9/src/loading.jl:159


MLJGLMInterface.LinearRegressor

In [72]:
Random.seed!(42)

TaskLocalRNG()

In [117]:
u_1 = 20
u_2 = u_3 = 1000

n_points = 100

dist_phi = Uniform(-5, 5)
phis_1 = rand(dist_phi, n_points)
phis_2 = rand(dist_phi, n_points)

intercept_1 = 16
intercept_2 = 30

slope_1 = 0.5
slope_2 = 4

std_dev = 1
noises_1 = rand(Normal(0, std_dev^2), n_points)
noises_2 = rand(Normal(0, std_dev^2), n_points)

demands_1 = intercept_1 .+ slope_1 .* phis_1 .+ noises_1
demands_2 = intercept_2 .+ slope_2 .* phis_2 .+ noises_2

df_1 = DataFrame(:intercept_1 => fill(1.0, n_points), :intercept_2 => fill(0.0, n_points), :phi => phis_1, :demand => demands_1)
df_2 = DataFrame(:intercept_1 => fill(0.0, n_points), :intercept_2 => fill(1.0, n_points), :phi => phis_2, :demand => demands_2)

df = vcat(df_1, df_2)
first(df, 5)

Row,intercept_1,intercept_2,phi,demand
,Float64,Float64,Float64,Float64
1,1.0,0.0,-3.28208,14.823
2,1.0,0.0,-2.92103,13.4813
3,1.0,0.0,4.09767,17.635
4,1.0,0.0,-1.70272,14.7649
5,1.0,0.0,-3.9125,14.6709


In [118]:
y, X = unpack(df, ==(:demand), colname -> true);

In [119]:
l_model = LinearW(fit_intercept=false)
l_mach = machine(l_model, X, y) 

fit!(l_mach);
l_model_int1, l_model_int2, l_model_slope = fitted_params(l_mach).coef

┌ Info: Training machine(LinearRegressor(fit_intercept = false, …), …).
└ @ MLJBase /home/luca/.julia/packages/MLJBase/kK4C2/src/machines.jl:492


3-element Vector{Float64}:
 16.57559740063439
 29.69089806678432
  2.2564331954342296

In [120]:
max_phi, min_phi = extrema(df.phi)
x_cap = [min_phi, max_phi]
y_cap = fill(u_1, length(x_cap));

y_l_model_com1 = l_model_int1 .+ l_model_slope .* x_cap
y_l_model_com2 = l_model_int2 .+ l_model_slope .* x_cap

2-element Vector{Float64}:
 40.96686171963628
 18.4745768946511

In [169]:
com1_weight = 5.0
weights = vcat(fill(com1_weight, n_points), fill(1.0, n_points))

w_model = LinearW(fit_intercept=false)
w_mach = machine(w_model, X, y, weights) 

fit!(w_mach);
w_model_int1, w_model_int2, w_model_slope = fitted_params(w_mach).coef

┌ Info: Training machine(LinearRegressor(fit_intercept = false, …), …).
└ @ MLJBase /home/luca/.julia/packages/MLJBase/kK4C2/src/machines.jl:492


3-element Vector{Float64}:
 16.242242182210667
 29.515871140084904
  1.0826422684173742

In [170]:
y_w_model_com1 = w_model_int1 .+ w_model_slope .* x_cap
y_w_model_com2 = w_model_int2 .+ w_model_slope .* x_cap

2-element Vector{Float64}:
 34.92610659151214
 24.13425229948844

In [192]:
p1 = plot([
    scatter(x=phis_1, y=demands_1, name="Commodity 1", mode="markers", line=attr(color="blue")), 
    scatter(x=phis_2, y=demands_2, name="Commodity 2", mode="markers", line=attr(color="orange")),
    scatter(x=x_cap, y=y_cap, name="Max capacity of arc AB", mode="lines", line=attr(color="red", dash="dot")),
    scatter(x=x_cap, y=y_l_model_com1, name="Predicted commodity 1", mode="lines", line=attr(color="blue")),
    scatter(x=x_cap, y=y_l_model_com2, name="Predicted commodity 2", mode="lines", line=attr(color="orange")),

    ],
    Layout(
        xaxis_title="Contextual information", 
        yaxis_title="Demand",
        showlegend=false,
        title="Unweighted observations"),
);

In [193]:
p2 = plot([
    scatter(x=phis_1, y=demands_1, name="Commodity 1", mode="markers", line=attr(color="blue"), showlegend=false), 
    scatter(x=phis_2, y=demands_2, name="Commodity 2", mode="markers", line=attr(color="orange"), showlegend=false),
    scatter(x=x_cap, y=y_cap, name="Max capacity of arc AB", mode="lines", line=attr(color="red", dash="dot"), showlegend=false),
    scatter(x=x_cap, y=y_w_model_com1, name="Predicted commodity 1", mode="lines", line=attr(color="blue"), showlegend=false),
    scatter(x=x_cap, y=y_w_model_com2, name="Predicted commodity 2", mode="lines", line=attr(color="orange"), showlegend=false),

    ],
    Layout(
        xaxis_title="Contextual information", 
        yaxis_title="Demand",
        title="Weighted observations (w1=5.0, w2=1.0)")
);


In [194]:
p = [p1 p2]
relayout!(p, title_text="Predicting demand for two commodities using a linear regression model with shared weights")
p

data: [
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [174]:
l_pred = Matrix(X) * fitted_params(l_mach).coef
w_pred = Matrix(X) * fitted_params(w_mach).coef

println("Unweighted RMSE: $(rms(l_pred, y))")
println("Weighted RMSE:   $(rms(w_pred, y))")

Unweighted RMSE: 5.187452398144874
Weighted RMSE:   6.179493790361813


In [175]:
function task_loss(pred, cap=u_1, arc_cost=1, n_points=n_points)
    com1_pred = pred[1:n_points]

    n = length(pred)
    n_above_cap = length(filter(p -> p > u_1, com1_pred))
    n_below_cap = n - n_above_cap

    return  arc_cost * (2 * n_above_cap + n_below_cap) / n
end

println("Unweighted optimization cost: $(task_loss(l_pred))")
println("Weighted optimization cost:   $(task_loss(w_pred))")


Unweighted optimization cost: 1.15
Weighted optimization cost:   1.07
